In [3]:
import pandas as pd
from gensim.parsing.preprocessing import remove_stopwords
from tqdm._tqdm_notebook import tqdm_notebook as tqdm
tqdm.pandas()

/tmp/ipykernel_509381/462869928.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook as tqdm


In [4]:
df_predictions = pd.read_csv("df_predictions.csv")

In [5]:
df = pd.read_csv("kidsInMindSubtitles2004.csv")
df.shape

(2004, 15)

In [2]:
import torch
from transformers import pipeline

# use dolly-v2-12b if you're using Colab Pro+, using pythia-2.8b for Free Colab
generate_text = pipeline(model="databricks/dolly-v2-2-8b", 
                         torch_dtype=torch.bfloat16, 
                         trust_remote_code=True,
                         device_map="auto")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [6]:
generate_text("Explain to me the difference between nuclear fission and fusion.")

[{'generated_text': 'Nuclear fission occurs when heavy elements like uranium or plutonium are disturbed into splitting into smaller pieces, a process known as a atomic bomb. This happens when a large amount of these elements are struck by incoming protons from another uranium nucleus. This process releases a great deal of energy.\n\nNuclear fusion, on the other hand, occurs when two or more light atoms like hydrogen, or deuterium and tritium, join together to form a heavier atom. This process releases much less energy than nuclear fission, but can create the energy necessary to power a fusion reactor. This can be accomplished because the resulting heavy atom is much more stable than its light counterparts.'}]

In [12]:
df_predictions['dolly_8b'] = None

In [21]:

for idx in tqdm(range(len(df_predictions))):
    
    query = "Given the subtitle rating of a movie = "+str(round(df_predictions['Predcited_Score'][idx], 1))+" \
    ; and a list of inappropriate words : "+ df_predictions['inappropriate_words'][idx] + " Generate \
    language summary like kids in mind website? Please consider the below two example language summaries of \
    Please consider the below two example language summaries of \
    movies:  \n 1." + df['Language_Description'][0] + "\n 2."+df['Language_Description'][1]
    
    query = query.replace("[", "")
    query = query.replace("]", "\n\n")
    
    
    response = generate_text(query)
    
    df_predictions['dolly_8b'][idx] = response[0]['generated_text']
    

  0%|          | 0/201 [00:00<?, ?it/s]

/tmp/ipykernel_509381/241509247.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['dolly_8b'][idx] = response[0]['generated_text']
/tmp/ipykernel_509381/241509247.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['dolly_8b'][idx] = response[0]['generated_text']
/tmp/ipykernel_509381/241509247.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_predictions['dolly_8b'][idx] = response[0]['generated

In [42]:
df_predictions.to_csv("df_predictions_dolly_8b.csv", index = False)

In [46]:
#df_predictions.head(2)

In [45]:
#df_predictions = pd.read_csv("df_predictions_dolly_8b.csv")

In [36]:
df_predictions['dolly_8b'][1]

'At least 1 f-word, 4 scatological terms, 4 anatomical terms, 3 mild obscenities, name-calling (freak, psycho, foolish, boring, stubborn, little man), 4 religious exclamations.'

In [37]:
df_predictions['language_summary'][1]

'About 25 F-words, 2 obscene hand gestures, 8 sexual references, 10 scatological terms, 13 anatomical terms, 23 mild obscenities, name-calling (dummy, psycho, freak show, retarded, dumb, jerk, idiot, "are you deaf or stupid," ugly), 17 religious exclamations.\n'

In [26]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [28]:
scores = scorer.score(df_predictions['language_summary'][0], df_predictions['dolly_8b'][0])

In [29]:
scores

{'rouge1': Score(precision=0.4166666666666667, recall=0.2631578947368421, fmeasure=0.3225806451612903),
 'rouge2': Score(precision=0.2, recall=0.125, fmeasure=0.15384615384615385),
 'rougeL': Score(precision=0.3333333333333333, recall=0.21052631578947367, fmeasure=0.2580645161290323)}

In [30]:

rouge1_scores = []
rouge2_scores = []
rougeL_scores = []

for idx in range(len(df_predictions)):
    
    scores = scorer.score(df_predictions['language_summary'][idx], df_predictions['dolly_8b'][idx])
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)


In [31]:
sum(rouge1_scores)/len(rouge1_scores)

0.17300255840188603

In [32]:
sum(rouge2_scores)/len(rouge2_scores)

0.07091939456329586

In [33]:
sum(rougeL_scores)/len(rougeL_scores)

0.14377770163453993

In [34]:
import numpy as np

In [35]:
sum(rougeL_scores)/len(rougeL_scores)

0.14377770163453993

In [38]:
import numpy as np

In [39]:
result_df = pd.DataFrame({"rougeL":[np.mean(rougeL_scores)]})

In [40]:
result_df

,rougeL
0,0.143778
